In [4]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
#import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math
import dill as pickle
from copy import deepcopy

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from mcmodels.core.utils import get_ccf_data

from mcmodels.core import VoxelModelCache
from mcmodels.connectivity.creation import get_connectivity_matrices3
from mcmodels.core.connectivity_data import get_connectivity_data
#from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures, get_leaves_ontologicalorder, get_indices_2ormore, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3
from mcmodels.regressors import NadarayaWatson
from mcmodels.core.plotting import plot_loss_surface,plot_loss_scatter
from mcmodels.models.expectedloss.crossvalidation import get_loss_surface_cv_spline
from mcmodels.models.expectedloss.crossvalidation import get_embedding_cv
from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loss, Crossval
from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists,CrossvalNW

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
FILE_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
OUTPUT_DIR = os.path.join(FILE_DIR, 'output')

input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#its unclear why the hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)


annotation/ccf_2017


In [6]:
#load version 3 list
ontological_order = get_ordered_summary_structures(cache,167587189)
#due to redundancy in ccfv3, remove 'MDRNv', 'MDRNd' from summary structures (include as leafs of MDRN)
ontological_order[277]
ontological_order = np.setdiff1d(ontological_order, [1098, 1107])
#load version 2 list
#ontological_order = get_ordered_summary_structures(cache,687527945)

mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}

#regionalize voxel model: compare with regional model
#regional parameters
cre = None
eid_set=None
high_res=False
threshold_injection = False

COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]

In [7]:
connectivity_data = get_connectivity_data(cache, major_structure_ids, experiments_exclude, remove_injection = False, structure_set_id=167587189)

connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()


512
703
1089
1097
315
313
354
698
771
803
477
549


In [8]:
connectivity_data.get_data_matrices(default_structure_ids)
connectivity_data.get_crelines(data_info)
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)
    
connectivity_data.ai_map = ai_map
connectivity_data.get_summarystructures(data_info)
connectivity_data.leafs = leafs
ontological_order_leaves = get_leaves_ontologicalorder(connectivity_data, ontological_order)
sid0 = list(connectivity_data.structure_datas.keys())[0]
eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
contra_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=1)
ipsi_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=2)
ipsi_target_regions, ipsi_target_counts = nonzero_unique(ipsi_targetkey, return_counts=True)
contra_target_regions, contra_target_counts = nonzero_unique(contra_targetkey, return_counts=True)
target_order = lambda x: np.array(ontological_order)[np.isin(ontological_order, x)]
permutation = lambda x: np.argsort(np.argsort(target_order(x)))
targ_ids = np.concatenate([ipsi_target_regions[permutation(ipsi_target_regions)],
                           contra_target_regions[permutation(contra_target_regions)]])

connectivity_data.get_regionalized_normalized_data(ontological_order, ipsi_targetkey, contra_targetkey)

above_thresh = {}
thresh = 1
for m in range(12):
    sid = major_structure_ids[m]
    above_thresh[sid] = np.where(connectivity_data.structure_datas[sid].reg_inj.sum(axis = 1) > thresh)[0]

In [15]:
from collections import Counter

In [49]:
# cts = Counter(np.concatenate(list(connectivity_data.creline.values())))
# #cts.most_common()
# #cts = Counter(np.concatenate(list(connectivity_data.leafs.values())))

# com_leaf = np.asarray(cts.most_common())
# ndf = np.asarray([[com_leaf[i,0] for i in range(100)], com_leaf[:100,1]]).transpose()
# pdf = pd.DataFrame(ndf)
# pdf = pdf.set_index(0)
# print(pdf.iloc[:20].to_latex())
# cts = Counter(np.concatenate(list(connectivity_data.leafs.values())))

# com_leaf = np.asarray(cts.most_common())
# ndf = np.asarray([[ia_map[com_leaf[i,0]] for i in range(100)], com_leaf[:100,1]]).transpose()
# pdf = pd.DataFrame(ndf)
# pdf = pdf.set_index(0)
# print(pdf.iloc[:20].to_latex())

In [50]:
#what leafs have the most cres
#Counter(connectivity_data.creline[315][np.where(connectivity_data.leafs[315] == 1015)[0]])

In [51]:
for sid in major_structure_ids:
    connectivity_data.structure_datas[sid].reg_proj_injnorm = connectivity_data.structure_datas[sid].reg_proj_injnorm[above_thresh[sid]]
    connectivity_data.structure_datas[sid].centroids = connectivity_data.structure_datas[sid].centroids[above_thresh[sid]]
    connectivity_data.creline[sid] = connectivity_data.creline[sid][above_thresh[sid]]
    connectivity_data.leafs[sid] = connectivity_data.leafs[sid][above_thresh[sid]]

In [52]:


gammas = np.asarray([0.0001,0.001,0.01,0.1,1,10,100,1000,10000])
#reg_proj_norm = {sid: connectivity_data.structure_datas[sid].reg_proj_norm for sid in major_structure_ids}
reg_proj_injnorm = {sid: connectivity_data.structure_datas[sid].reg_proj_injnorm for sid in major_structure_ids}
#sum2_index_matrices = {sid: connectivity_data.sum2_index_matrices[sid] for sid in major_structure_ids} 
#cresum2_evalindices = {sid: connectivity_data.cresum2_evalindices[sid] for sid in major_structure_ids} 
#cresum2_leaf2_evalindices = {sid: connectivity_data.cresum2_leaf2_evalindices[sid] for sid in major_structure_ids} 
crelines = {sid: connectivity_data.creline[sid] for sid in major_structure_ids}
#look at visp6a prediction accuracy
#lets just make the eval plots
connectivity_data.leaf2_index_matrices = get_indices_2ormore(connectivity_data.leafs)
connectivity_data.get_creleaf_combos()
connectivity_data.creleaf2_index_matrices = get_indices_2ormore(connectivity_data.creleaf_combos)
connectivity_data.get_creleaf_combos()
connectivity_data.cre2_index_matrices = get_indices_2ormore(crelines)

#connectivity_data.cresum2_evalindices = get_eval_indices(connectivity_data.cresum2_index_matrices)
connectivity_data.creleaf2_evalindices = get_eval_indices(connectivity_data.creleaf2_index_matrices)
creleaf2_evalindices = {sid: connectivity_data.creleaf2_evalindices[sid] for sid in major_structure_ids} 
cre2_index_matrices = {sid: connectivity_data.cre2_index_matrices[sid] for sid in major_structure_ids} 
#connectivity_data.cresum2_leaf2_evalindices = get_eval_indices(connectivity_data.cresum2_leaf2_index_matrices)


connectivity_data.major2_index_matrices = {}
for sid in major_structure_ids:
     connectivity_data.major2_index_matrices[sid] =  np.expand_dims(np.ones(crelines[sid].shape),0)
major2_index_matrices = {sid:connectivity_data.major2_index_matrices[sid] for sid in major_structure_ids}

wtmajor2_index_matrices = {}
for sid in major_structure_ids:
    wtm = np.zeros(major2_index_matrices[sid].shape) 
    #print(wtm.shape)
    wtmajor2_index_matrices[sid] = wtm[:,np.where(crelines[sid] == 'C57BL/6J')[0]] = 1
#get_indices2(connectivity_data.creline[sid], np.asarray(['C57BL/6J']),leafs[sid])

#ncresum2_leaf2 = np.zeros(12)
#ncresum2 = np.zeros(12)
ncreleaf2 = np.zeros(12)
ntotal = np.zeros(12)
for i in range(12):
    sid = major_structure_ids[i]
    ntotal[i] = connectivity_data.structure_datas[sid].reg_proj.shape[0]
    #ncresum2_leaf2[i] =len(cresum2_leaf2_evalindices[sid])
    #ncresum2[i] = len(cresum2_evalindices[sid])
    ncreleaf2[i] = len(creleaf2_evalindices[sid])

major_structures = np.asarray(major_structures)
reo = np.asarray([4,7,2,1,10,9,11,3,5,8,6,0], dtype = int)
df = pd.DataFrame([ntotal,  ncreleaf2], dtype = int, index=['Total','Cre-Leaf'], columns = major_structures[reo]).transpose()
#df = pd.DataFrame([ntotal, ncresum2, ncresum2_leaf2,  ncreleaf2], dtype = int, index=['Total','Cre-Summary','Cre-Summary, Leaf','Cre-Leaf']).transpose()
print(df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  Total &  Cre-Leaf \\
\midrule
Isocortex &     36 &         4 \\
OLF       &      7 &         2 \\
HPF       &    122 &        59 \\
CTXsp     &     85 &        38 \\
STR       &   1128 &       709 \\
PAL       &     68 &        18 \\
TH        &     46 &         7 \\
HY        &     35 &        17 \\
MB        &     33 &         8 \\
P         &     30 &        11 \\
MY        &     78 &        44 \\
CB        &     83 &        29 \\
\bottomrule
\end{tabular}



In [53]:
print(connectivity_data.structure_datas[sid].reg_proj_injnorm.shape)
print(connectivity_data.structure_datas[sid].centroids.shape)
print(connectivity_data.creline[sid].shape)
print(connectivity_data.creline[sid].shape)
#frac_learn[m]

(82, 621)
(82, 3)
(82,)
(82,)


In [54]:
#get surfaces leaf
frac_learn = np.ones(12)
for m in range(12):
    sid = major_structure_ids[m]
    connectivity_data.structure_datas[sid].loss_surface_cv_leaf = get_loss_surface_cv_spline(connectivity_data.structure_datas[sid].reg_proj_injnorm,
                                                                                                 connectivity_data.structure_datas[sid].centroids,
                                                                                                 connectivity_data.creline[sid],
                                                                                                 connectivity_data.leafs[sid],
                                                                                                 frac_learn[m])
    connectivity_data.structure_datas[sid].smoothed_losses_leaf = get_embedding_cv(surface = connectivity_data.structure_datas[sid].loss_surface_cv_leaf,
                                                                              dists = pairwise_distances(connectivity_data.structure_datas[sid].centroids)**2,
                                                                              cre_distances_cv = connectivity_data.structure_datas[sid].loss_surface_cv_leaf.cre_distances_cv)
    

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/expectedloss/shapeconstrained.py:47: RuntimeWarning: invalid value encountered in true_divide
  coordinates_normed = coordinates / np.linalg.

In [55]:
# #what are the indices that have both 2 in the same leaf, and 2 of the same cre in the sum
# connectivity_data.cresum2_leaf2_index_matrices = deepcopy(connectivity_data.cresum2_index_matrices)
# for sid in major_structure_ids:
#     included_leaf2 =  connectivity_data.leaf2_index_matrices[sid].sum(axis = 0)
#     included_cresum2 = connectivity_data.cresum2_index_matrices[sid].sum(axis = 0)
#     keepers = np.intersect1d( np.where(included_leaf2 == 1)[0], np.where(included_cresum2 == 1)[0])
#     #print(len(keepers))
#     setdiff = np.setdiff1d(list(range(connectivity_data.leaf2_index_matrices[sid].shape[1])), keepers)
#     connectivity_data.cresum2_leaf2_index_matrices[sid][:,setdiff] = 0
    

In [56]:
leaf2_index_matrices = {sid: connectivity_data.leaf2_index_matrices[sid] for sid in major_structure_ids} 
distances = {sid: connectivity_data.structure_datas[sid].smoothed_losses_leaf for sid in major_structure_ids}
twostage_leaf_creleaf2 = CrossvalNW(reg_proj_injnorm, distances, leaf2_index_matrices, creleaf2_evalindices,gammas)
twostage_leaf_creleaf2.gammas = gammas #ds
twostage_leaf_creleaf2.predictions = twostage_leaf_creleaf2.get_predictions()
twostage_leaf_creleaf2.get_results_loocv()
twostage_leaf_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)

In [57]:
twostage_leaf_creleaf2.meanloss

array([0.49551171, 0.49739986, 0.31802197, 0.40858308, 0.37257939,
       0.37779774, 0.49411563, 0.3871113 , 0.56212459, 0.44327197,
       0.29837562, 0.53393518])

In [58]:
creleaf2_index_matrices = {sid: connectivity_data.creleaf2_index_matrices[sid] for sid in major_structure_ids} 
leaf2_index_matrices = {sid: connectivity_data.leaf2_index_matrices[sid] for sid in major_structure_ids} 
pds = {}
for sid in major_structure_ids:
    pds[sid] = pairwise_distances(connectivity_data.structure_datas[sid].centroids)**2
distances = pds

##{sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}
nw_major_creleaf2 = CrossvalNW(reg_proj_injnorm, distances,major2_index_matrices, creleaf2_evalindices,gammas)
nw_major_creleaf2.gammas = gammas
nw_major_creleaf2.predictions = nw_major_creleaf2.get_predictions()
nw_major_creleaf2.get_results_loocv()
nw_major_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/voxel/crossvalidation.py:43: RuntimeWarning: overflow encountered in exp
  weights_i = np.exp(-dists_i * gamma)  # weights[i,:] / np.nansum(weights[i,:][otherindices])


In [59]:
nw_major_creleaf2.meanloss

array([0.61503358, 0.49739986, 0.4904136 , 0.46239138, 0.49748706,
       0.30349243, 0.65441012, 0.51575293, 0.76407003, 0.46641033,
       0.42060867, 0.52162145])

In [60]:
nw_majorwt_creleaf2 = CrossvalNW(reg_proj_injnorm, distances,wtmajor2_index_matrices, creleaf2_evalindices,gammas)
nw_majorwt_creleaf2.gammas = gammas
nw_majorwt_creleaf2.predictions = nw_major_creleaf2.get_predictions()
nw_majorwt_creleaf2.get_results_loocv()
nw_majorwt_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


In [61]:
nw_majorwt_creleaf2.meanloss

array([0.61503358, 0.49739986, 0.4904136 , 0.46239138, 0.49748706,
       0.30349243, 0.65441012, 0.51575293, 0.76407003, 0.46641033,
       0.42060867, 0.52162145])

In [62]:
nw_leaf_creleaf2 = CrossvalNW(reg_proj_injnorm, distances,leaf2_index_matrices, creleaf2_evalindices,gammas)
nw_leaf_creleaf2.gammas = gammas# ds
nw_leaf_creleaf2.predictions = nw_leaf_creleaf2.get_predictions()
nw_leaf_creleaf2.get_results_loocv()
nw_leaf_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


In [63]:
nw_leaf_creleaf2.meanloss

array([0.50036524, 0.49739986, 0.41910314, 0.45060663, 0.49744626,
       0.32599315, 0.49491104, 0.41232511, 0.75809955, 0.44488629,
       0.35746546, 0.50139   ])

In [64]:
#distances = pds#{sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}
nw_creleaf_creleaf2 = CrossvalNW(reg_proj_injnorm, distances,creleaf2_index_matrices, creleaf2_evalindices,gammas)
nw_creleaf_creleaf2.gammas = gammas# ds
nw_creleaf_creleaf2.predictions = nw_creleaf_creleaf2.get_predictions()
nw_creleaf_creleaf2.get_results_loocv()
nw_creleaf_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


In [65]:
nw_creleaf_creleaf2.meanloss

array([0.84935969, 0.49739986, 0.32463493, 0.4297395 , 0.38226119,
       0.35441512, 0.67832154, 0.41229759, 0.56213315, 0.539072  ,
       0.3283235 , 0.72395377])

In [24]:


# nw_sum_cresum2leaf2 = CrossvalNW(reg_proj_norm, distances,sum2_index_matrices, cresum2_leaf2_evalindices,gammas)
# nw_sum_cresum2leaf2.gammas = gammas
# nw_sum_cresum2leaf2.predictions = nw_sum_cresum2leaf2.get_predictions()
# nw_sum_cresum2leaf2.get_results_loocv()
# nw_sum_cresum2leaf2.get_results_weightedloocv(connectivity_data.summary_structures,crelines,ia_map)
#nw_sum_creleaf2.meanloss





/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/voxel/crossvalidation.py:43: RuntimeWarning: overflow encountered in exp
  weights_i = np.exp(-dists_i * gamma)  # weights[i,:] / np.nansum(weights[i,:][otherindices])


In [66]:
#distances = pds#{sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}
nw_cremajor_creleaf2 = CrossvalNW(reg_proj_injnorm, distances,cre2_index_matrices, creleaf2_evalindices,gammas)
nw_cremajor_creleaf2.gammas = gammas# ds
nw_cremajor_creleaf2.predictions = nw_cremajor_creleaf2.get_predictions()
nw_cremajor_creleaf2.get_results_loocv()
nw_cremajor_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


In [67]:
nw_cremajor_creleaf2.meanloss

array([0.68923929, 0.49739986, 0.4710937 , 0.45278232, 0.40968429,
       0.35290677, 0.58946796, 0.434511  , 0.56213231, 0.52581669,
       0.35270877, 0.72261032])

In [68]:

mean_gammas = np.ones(12)*0.000001
mean_creleaf_creleaf2 = CrossvalNW(reg_proj_injnorm, distances, creleaf2_index_matrices, creleaf2_evalindices,mean_gammas)
mean_creleaf_creleaf2.gammas = mean_gammas# ds
mean_creleaf_creleaf2.predictions = mean_creleaf_creleaf2.get_predictions()
mean_creleaf_creleaf2.get_results_loocv()
mean_creleaf_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


In [69]:
mean_creleaf_creleaf2.meanloss

array([0.84935969, 0.49739986, 0.32799569, 0.42973161, 0.39058528,
       0.42904373, 0.7789465 , 0.41798414, 0.56213315, 0.53905097,
       0.33089021, 0.72395148])

In [70]:
#el_labels = np.zeros(150, dtype = str)
models = np.asarray(['Mean','NW','NW','NW','NW','NW','EL'])
datas = np.asarray([r"$I_c \cap I_L$",
                   r"$I_c \cap I_M$",
                   r"$I_c \cap I_L$",
                    r"$I_L$",
                  r"$I_{wt} \cap I_M$",
                  r"$I_M$"  ,
                  r"$I_L$"])
multi_ind = np.vstack([models,datas])
multi_ind = np.asarray(multi_ind, dtype = str)
multi_ind = list(zip(*multi_ind))
multi_ind = pd.MultiIndex.from_tuples(multi_ind, names=[r"$\widehat f$",r"$\mathcal D$"])

reo = np.asarray([4,7,2,1,10,9,11,3,5,8,6,0])
results = pd.DataFrame([mean_creleaf_creleaf2.meanloss_weighted,
nw_cremajor_creleaf2.meanloss_weighted,
nw_creleaf_creleaf2.meanloss_weighted,
nw_leaf_creleaf2.meanloss_weighted,
nw_majorwt_creleaf2.meanloss_weighted,
nw_major_creleaf2.meanloss_weighted,
twostage_leaf_creleaf2.meanloss_weighted], index = multi_ind, columns = major_structures).transpose()#, cre2_index_matrices
trunc = lambda x: math.trunc(1000 * x) / 1000;

results = results.applymap(trunc)

In [71]:
print(results.iloc[reo].to_latex(index=True,escape=False))
#{sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}


\begin{tabular}{lrrrrrrr}
\toprule
$\widehat f$ &           Mean & \multicolumn{5}{l}{NW} &     EL \\
$\mathcal D$ & $I_c \cap I_L$ & $I_c \cap I_M$ & $I_c \cap I_L$ &  $I_L$ & $I_{wt} \cap I_M$ &  $I_M$ &  $I_L$ \\
\midrule
Isocortex &          0.413 &          0.453 &          0.408 &  0.538 &             0.528 &  0.528 &  0.396 \\
OLF       &          0.499 &          0.504 &          0.494 &  0.441 &             0.543 &  0.543 &  0.437 \\
HPF       &          0.336 &          0.483 &          0.332 &  0.444 &             0.501 &  0.501 &  0.321 \\
CTXsp     &          0.497 &          0.497 &          0.497 &  0.497 &             0.497 &  0.497 &  0.497 \\
STR       &          0.359 &          0.386 &          0.359 &  0.364 &             0.433 &  0.433 &  0.322 \\
PAL       &          0.519 &          0.497 &          0.519 &  0.436 &             0.459 &  0.459 &  0.434 \\
TH        &          0.769 &          0.767 &          0.769 &  0.514 &             0.539 &  0.539 &  0.556 \